In [40]:
import urllib
import pandas as pd
import time
from tqdm.notebook import tqdm
import re
import wget
import numpy as np

In [2]:
from lxml import html
from random import randint
from random import sample
from time import sleep
import requests
import urllib.request
from urllib.request import urlopen

# extract doi list

In [37]:
all_abstracts = pd.read_csv('data/included_abstracts.csv', index_col = 0)
len(all_abstracts)

42587

In [38]:
all_abstracts['article_date'] = pd.to_datetime(all_abstracts['article_date'])
decade_df = all_abstracts[(all_abstracts['article_date'] > '2012-01-01') & (all_abstracts['article_date'] <'2022-01-01')]
len(decade_df)

28703

In [39]:
decade_df.sort_values(by='article_date',  ascending=False, inplace=True)
decade_df.head(50)

C:\Users\Joe Z\anaconda3\envs\scraper\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


pmid                               doi  \
23767  35011970               10.3390/jcm11010229   
34412  34972109      10.1371/journal.pone.0261698   
29263  34971557      10.1371/journal.pone.0260600   
26000  35005197                10.1002/dad2.12264   
39158  34969093            10.1093/cercor/bhab474   
42411  35005676       10.1016/j.cvdhj.2021.12.003   
18320  35008361           10.3390/cancers14010188   
542    35003258              10.1155/2021/5499385   
20236  34998166        10.1016/j.cmpb.2021.106613   
16453  35011959               10.3390/jcm11010219   
5056   34972768       10.1136/bmjopen-2021-053808   
20668  34979435    10.1016/j.ijmedinf.2021.104669   
4043   34911053          10.1088/1361-6560/ac4348   
26236  34971530          10.1109/TMI.2021.3139999   
17694  35009845                 10.3390/s22010299   
38246  35045383      10.1016/j.neunet.2021.12.014   
41228  35003245              10.1155/2021/5759184   
22402  35009825                 10.3390/s22010283   
36450  34633928        10.1109/TUFFC.2021.3119251   
5226   34905733          10.1088/1361-6560/ac4316   
13921  34979213      10.1016/j.radonc.2021.12.033   
16895  35009834                 10.3390/s22010292   
25067  34998220  10.1016/j.compbiomed.2021.105192   
36908  34460373        10.1109/TUFFC.2021.3109117   
32321  35005207                10.1002/trc2.12229   
40167  34879437                10.1111/jcpe.13574   
1376   34971441         10.1007/s11596-021-2501-4   
39258  34972151      10.1371/journal.pone.0261433   
41541  34979129        10.1016/j.apmr.2021.12.006   
20242  35033757        10.1016/j.cmpb.2021.106612   
839    35111273            10.2478/joeb-2021-0021   
14918  35144199       10.1016/j.media.2021.102329   
33514  34973967         10.1016/j.gie.2021.12.020   
34512  34965805         10.1177/03009858211067478   
9061   34969378        10.1186/s12911-021-01731-3   
40937  34973966         10.1016/j.gie.2021.12.019   
37408  35005017              10.1155/2021/5047501   
17612  35036353          10.3389/fonc.2021.725320   
16937  35011934               10.3390/jcm11010192   
37268  35002476        10.1007/s11280-021-00983-3   
29168  34970824                10.1002/jmri.28047   
22409  35062267                 10.3390/v14010063   
42282  34968146       10.1097/JU.0000000000002390   
17770  35009794                 10.3390/s22010253   
36840  34460371          10.1109/TMI.2021.3108949   
10249  34965817     10.1080/17538157.2021.2019038   
25079  34460370          10.1109/TMI.2021.3108910   
19189  34491896          10.1109/TMI.2021.3110829   
23163  35052094                 10.3390/e24010068   
17384  35009788                 10.3390/s22010245   

                                                   title  \
23767  An Application of Machine Learning That Uses t...   
34412  DAVS-NET: Dense Aggregation Vessel Segmentatio...   
29263  Optical coherence tomography for identificatio...   
26000  Deep learning improves utility of tau PET in t...   
39158  Predicting Superagers by Machine Learning Clas...   
42411  Identifying Risk of Adverse Outcomes in COVID-...   
18320  A Clinical Prognostic Model Based on Machine L...   
542    Deep Learning to Predict EGFR Mutation and PD-...   
20236  TMSF-Net: Multi-series fusion network with tre...   
16453  Machine Learning to Calculate Heparin Dose in ...   
5056   Using natural language processing to extract s...   
20668  Deep transfer learning based model for colorec...   
4043   Automated segmentation of metal stent and bior...   
26236  Shadow-consistent Semi-supervised Learning for...   
17694  Machine Learning Methods for Automatic Silent ...   
38246  Evolved explainable classifications for lymph ...   
41228  Machine Learning Implementation of a Diabetic ...   
22402  Multi-Task Model for Esophageal Lesion Analysi...   
36450  Bi-Modal Transfer Learning for Classifying Bre...   
5226   Weakly-supervised lesion analysis with a CNN-b...   
13921  Computer-aided segmentation on MRI for p

In [40]:
scrape_df = decade_df[['pmid', 'doi', 'title', 'abstract']].copy()
scrape_df.to_csv('data/decade_abstracts.csv')

In [41]:
id_dict = dict(zip(scrape_df['pmid'], scrape_df['doi']))
pmid_list = scrape_df['pmid'].tolist()
doi_list = scrape_df['doi'].tolist()

# create urls for science direct api

In [42]:
api_key = "8218e9fd14ed4e6e6a2f2c4b602d3e40"

In [43]:
url_list = []

for doi in doi_list:
    #try:
    #    doi_encoded = urllib.parse.quote(doi, safe='')
    #except:
    #    doi_encoded = ("nan")
    link = "https://api.elsevier.com/content/article/doi/{}?APIKey={}".format(doi, api_key)
    #link = "https://api.elsevier.com/content/article/doi/{}?APIKey={}".format(doi_encoded, api_key)
    
    url_list.append(link)

len(url_list)

28703

# elsevier science direct api

In [44]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36'}
text_block = []

for url in tqdm(url_list):
    
    sleep(randint(1,3))
    
    page = requests.get(url, headers=headers)
    tree = html.fromstring(page.content)
    
    ## drop reference elements
    for reference in tree.xpath('//body//section//cross-ref'):
        reference.drop_tree()
    
    for reference in tree.xpath('//body//section//cross-refs'):
        reference.drop_tree()
    
    for reference in tree.xpath('//body//section//inter-ref'):
        reference.drop_tree()
    
    body_text = tree.xpath('//body//section/descendant-or-self::*/text()')
    
    text_block.append(body_text)
    

  0%|          | 0/28703 [00:00<?, ?it/s]

In [45]:
scrape_df['science_direct_xml'] = text_block

# join each xml list to string for export 
(allows re-separation into list in future for cleaning)

In [63]:
string_block = []

for item in tqdm(text_block):
    string = '[list_separator]'.join([x for x in item])
    
    string_block.append(string)
    
scrape_df['joined_string_xml'] = string_block

  0%|          | 0/28703 [00:00<?, ?it/s]

In [64]:
scrape_df.head(50)

pmid                               doi  \
23767  35011970               10.3390/jcm11010229   
34412  34972109      10.1371/journal.pone.0261698   
29263  34971557      10.1371/journal.pone.0260600   
26000  35005197                10.1002/dad2.12264   
39158  34969093            10.1093/cercor/bhab474   
42411  35005676       10.1016/j.cvdhj.2021.12.003   
18320  35008361           10.3390/cancers14010188   
542    35003258              10.1155/2021/5499385   
20236  34998166        10.1016/j.cmpb.2021.106613   
16453  35011959               10.3390/jcm11010219   
5056   34972768       10.1136/bmjopen-2021-053808   
20668  34979435    10.1016/j.ijmedinf.2021.104669   
4043   34911053          10.1088/1361-6560/ac4348   
26236  34971530          10.1109/TMI.2021.3139999   
17694  35009845                 10.3390/s22010299   
38246  35045383      10.1016/j.neunet.2021.12.014   
41228  35003245              10.1155/2021/5759184   
22402  35009825                 10.3390/s22010283   
36450  34633928        10.1109/TUFFC.2021.3119251   
5226   34905733          10.1088/1361-6560/ac4316   
13921  34979213      10.1016/j.radonc.2021.12.033   
16895  35009834                 10.3390/s22010292   
25067  34998220  10.1016/j.compbiomed.2021.105192   
36908  34460373        10.1109/TUFFC.2021.3109117   
32321  35005207                10.1002/trc2.12229   
40167  34879437                10.1111/jcpe.13574   
1376   34971441         10.1007/s11596-021-2501-4   
39258  34972151      10.1371/journal.pone.0261433   
41541  34979129        10.1016/j.apmr.2021.12.006   
20242  35033757        10.1016/j.cmpb.2021.106612   
839    35111273            10.2478/joeb-2021-0021   
14918  35144199       10.1016/j.media.2021.102329   
33514  34973967         10.1016/j.gie.2021.12.020   
34512  34965805         10.1177/03009858211067478   
9061   34969378        10.1186/s12911-021-01731-3   
40937  34973966         10.1016/j.gie.2021.12.019   
37408  35005017              10.1155/2021/5047501   
17612  35036353          10.3389/fonc.2021.725320   
16937  35011934               10.3390/jcm11010192   
37268  35002476        10.1007/s11280-021-00983-3   
29168  34970824                10.1002/jmri.28047   
22409  35062267                 10.3390/v14010063   
42282  34968146       10.1097/JU.0000000000002390   
17770  35009794                 10.3390/s22010253   
36840  34460371          10.1109/TMI.2021.3108949   
10249  34965817     10.1080/17538157.2021.2019038   
25079  34460370          10.1109/TMI.2021.3108910   
19189  34491896          10.1109/TMI.2021.3110829   
23163  35052094                 10.3390/e24010068   
17384  35009788                 10.3390/s22010245   

                                                   title  \
23767  An Application of Machine Learning That Uses t...   
34412  DAVS-NET: Dense Aggregation Vessel Segmentatio...   
29263  Optical coherence tomography for identificatio...   
26000  Deep learning improves utility of tau PET in t...   
39158  Predicting Superagers by Machine Learning Clas...   
42411  Identifying Risk of Adverse Outcomes in COVID-...   
18320  A Clinical Prognostic Model Based on Machine L...   
542    Deep Learning to Predict EGFR Mutation and PD-...   
20236  TMSF-Net: Multi-series fusion network with tre...   
16453  Machine Learning to Calculate Heparin Dose in ...   
5056   Using natural language processing to extract s...   
20668  Deep transfer learning based model for colorec...   
4043   Automated segmentation of metal stent and bior...   
26236  Shadow-consistent Semi-supervised Learning for...   
17694  Machine Learning Methods for Automatic Silent ...   
38246  Evolved explainable classifications for lymph ...   
41228  Machine Learning Implementation of a Diabetic ...   
22402  Multi-Task Model for Esophageal Lesion Analysi...   
36450  Bi-Modal Transfer Learning for Classifying Bre...   
5226   Weakly-supervised lesion analysis with a CNN-b...   
13921  Computer-aided segmentation on MRI for p

In [65]:
# scrape_df.to_csv('output/01a_science_direct_xml_raw.csv')

# clean xml output

In [14]:
raw_df = pd.read_csv('output/01a_science_direct_xml_raw.csv', index_col = 0)

In [15]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28703 entries, 23767 to 21081
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                28703 non-null  int64 
 1   doi                 27487 non-null  object
 2   title               28702 non-null  object
 3   abstract            28703 non-null  object
 4   science_direct_xml  28703 non-null  object
 5   joined_string_xml   6127 non-null   object
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


In [16]:
scrape_df = raw_df[~raw_df['joined_string_xml'].isnull()]

In [20]:
scrape_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6127 entries, 42411 to 21081
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                6127 non-null   int64 
 1   doi                 6127 non-null   object
 2   title               6126 non-null   object
 3   abstract            6127 non-null   object
 4   science_direct_xml  6127 non-null   object
 5   joined_string_xml   6127 non-null   object
dtypes: int64(1), object(5)
memory usage: 335.1+ KB


In [17]:
string_xml = scrape_df['joined_string_xml'].to_list()
len(string_xml)

6127

In [18]:
direct_xml = []

for joined in string_xml:
    try:
        xml = joined.split('[list_separator]')
        direct_xml.append(xml)
    except:
        direct_xml.append('')

len(direct_xml)

6127

In [19]:
direct_xml[0]

['\n                  ',
 'Introduction',
 '\n                  ',
 'Coronavirus disease 2019 (COVID-19) has now been documented in at least 230 million people worldwide, resulting in at least 4.7 million deaths. This affliction has been linked with a range of cardiovascular complications including arrhythmias, myocarditis, acute coronary syndrome, and thromboembolism. These ',
 'de novo',
 ' cardiovascular events, as well as pre-existing cardiovascular comorbidity, are linked to adverse outcomes in patients with COVID-19. Work in cardiomyocytes derived from human pluripotent stem cells showed that SARS-CoV-2 infection impaired electrophysiological and contractile function and led to widespread cell death, suggesting cardiovascular symptoms may be a direct consequence of cardiotoxicity. Atrial fibrillation (AF) is associated with increased mortality in infected patients, especially when it newly presents during hospitalization for COVID-19. Biomarker-detected myocardial injury, thrombo

In [21]:
## conditions

def alphachars(x): return sum(a.isalpha() for a in x) > 4
def charchar(x): return bool(re.match(r".*[a-zA-Z][a-zA-Z].*", x)) == True
def istitle(x): return ((('method' in x) or ('material' in x) or ('result' in x) or ('discussion' in x) or ('conclusion' in x)
                        or ('references' in x) or ('bibliography' in x) or ('data' in x)) 
                        and (sum(l.isalpha() for l in x) < 30))
def istextblock(x): return sum(l.isalpha() for l in x) > 100
#def methodtitle(x): return ((('method' in x)) or ('material' in x) and (sum(l.isalpha() for l in x)<30) and ('supplement' not in x)) 
def methodtitle(x): return ((('method' in x) or ('data' in x) or ('material' in x)) and (sum(l.isalpha() for l in x)<30)) 
## symbol cleaning

def lbracketcount(x): return x.count('{') < 4
def rbracketcount(x): return x.count('}') < 4
def tagcount(x): return x.count('"."') < 4
def hashcount(x): return x.count('#') < 4
def divcount(x): return x.count('</') < 4
def linecount(x): return x.count('||') < 4

In [22]:
pmid_list = []
methods_list = []

for base_text in tqdm(direct_xml):

    keep_text = []
    cull_text = []
    method_candidates = []

    try:
        ## (5) remove line breaks and extra spaces
        base_text = [x.replace('\n', ' ') for x in base_text]
        base_text = [x.replace('    ', ' ') for x in base_text]
        base_text = [x.replace('   ', ' ') for x in base_text]
        base_text = [x.replace('  ', ' ') for x in base_text]
        base_text = [x.lower() for x in base_text]
        
        ## (6) keep list elements if conforms to conditions, return 'cull_text' that is much cleaner
        for x in base_text:
            if (charchar(x) and alphachars(x)) and (istitle(x) or istextblock(x)):
                keep_text.append(x)
            else:
                continue

        for x in keep_text:
            if (lbracketcount(x) and rbracketcount(x) and tagcount(x) and hashcount(x) and divcount(x) and linecount(x)):
                cull_text.append(x)
            else:
                continue 
            
        ## (7) find method title and method section
        
        for i, x in enumerate(cull_text):    
            method_following = [] #list of all elements following and including methods title
            method_only = [] #list of all elements inbetween 'method' title and next section title

            if methodtitle(x) == True:
                method_following = cull_text[i:] 
                
                for i, x in enumerate(method_following):
                    if 'result' in x and sum(l.isalpha() for l in x) <  30:
                        #print("result title!")
                        method_only = method_following[:i]
                        break
                    elif 'discussion' in x and sum(l.isalpha() for l in x) <  30:
                        #print("discussion title!")
                        method_only = method_following[:i]                
                        break
                    elif 'conclusion' in x and sum(l.isalpha() for l in x) <  30:
                        #print("conclusion title!")
                        method_only = method_following[:i]               
                        break
                    elif 'references' in x and sum(l.isalpha() for l in x) <  30:
                        #print("reference title!")
                        method_only = method_following[:i]            
                        break
                    elif 'bibliography' in x and sum(l.isalpha() for l in x) <  30:
                        #print("reference title!")
                        method_only = method_following[:i]                 
                        break                      
                    else:
                        method_only = method_following #if none of these are found to end text block, then take the whole text block
    
                method_string = ' '.join(method_only)        
    
                if istextblock(method_string):
                    method_candidates.append(method_string)
                else:
                    continue
                
            else:
                continue
        
        print(len(method_candidates))    
        methods_list.append(method_candidates) # each paper now has a list of method candidates (each is a string)

    except:
        print('fail')
        methods_list.append('fail')
        continue            

print(len(methods_list))

  0%|          | 0/6127 [00:00<?, ?it/s]

2
2
2
1
2
3
2
1
1
4
1
2
2
2
2
1
2
3
4
3
0
1
1
2
2
1
6
3
4
2
1
3
4
3
3
2
6
4
2
1
1
3
2
2
3
4
7
3
2
4
1
3
14
1
3
3
2
1
3
3
2
2
4
6
2
2
2
3
1
2
4
2
3
5
1
2
2
2
7
2
1
5
4
3
2
1
2
5
1
1
3
3
2
4
1
2
2
1
4
4
3
1
2
3
1
2
5
1
3
2
2
3
1
2
2
7
2
1
1
1
6
2
0
1
2
1
3
6
2
2
3
2
3
2
2
3
1
1
2
4
3
2
2
1
2
3
5
4
4
3
3
3
8
2
2
1
1
1
4
1
4
2
1
5
3
2
3
1
2
3
4
3
1
4
2
5
2
4
2
7
2
2
2
4
1
1
5
2
1
1
1
3
2
3
2
4
3
1
10
4
5
2
3
4
3
4
5
2
2
3
1
4
8
1
3
2
2
6
4
3
1
1
4
9
2
3
2
3
2
3
2
2
2
7
1
2
3
3
12
3
2
1
9
3
1
3
3
2
1
2
2
1
3
3
4
3
4
1
2
2
6
4
1
1
2
1
3
2
2
1
1
2
3
3
2
2
3
4
6
1
2
1
2
5
3
1
3
2
4
4
2
5
2
2
1
3
3
1
2
5
1
2
4
2
4
4
1
2
2
8
2
8
2
1
1
8
2
3
2
1
1
6
1
9
3
25
1
1
2
2
3
2
1
4
3
3
2
3
4
2
6
2
4
2
2
1
2
3
1
2
1
6
2
1
1
3
4
4
7
1
2
3
2
2
2
4
1
5
2
4
1
1
5
2
3
5
3
4
4
4
2
2
4
2
2
4
2
5
3
7
4
2
2
1
2
3
35
10
4
2
5
3
2
1
1
2
6
7
1
3
1
3
4
4
1
4
1
1
2
8
10
2
7
1
2
2
1
2
3
9
1
2
4
7
1
1
3
6
4
1
1
2
1
3
4
3
4
2
2
3
1
2
3
1
0
6
1
5
1
3
1
1
17
2
6
2
2
2
2
4
2
2
5
2
3
5
7
2
7
4
2
3
4
3
3
3
1
2
1
2
6
4
2
3
1
4


0
2
5
2
4
5
3
3
2
2
2
4
2
2
2
2
11
4
2
2
0
1
2
4
2
3
2
4
3
2
2
3
2
1
7
1
3
3
5
1
1
2
2
2
1
1
2
0
7
2
2
1
2
1
8
4
3
7
1
1
1
5
2
2
6
7
1
1
1
1
1
1
1
1
2
3
4
3
8
2
1
1
1
2
2
3
1
3
2
2
3
3
3
2
6
2
3
1
2
2
2
3
2
3
1
3
3
5
2
9
2
2
2
2
1
2
3
4
2
6
2
4
2
3
2
1
1
3
0
0
4
1
7
3
1
1
4
1
7
1
3
2
5
3
3
3
1
4
6
3
3
2
4
2
2
0
1
4
0
6
2
2
0
2
1
1
2
2
1
2
2
4
2
1
1
2
2
2
2
1
5
2
3
15
1
4
1
2
4
4
3
2
2
2
2
1
1
4
4
4
1
1
3
2
1
2
3
1
2
3
3
1
2
1
3
4
2
1
4
1
4
3
1
1
2
1
1
7
1
4
2
2
2
2
3
4
2
2
4
1
1
1
2
10
4
2
4
3
9
2
2
2
1
2
3
1
1
2
2
2
1
1
2
1
1
2
1
5
2
3
2
1
1
1
1
1
1
1
1
4
7
2
3
1
4
1
4
2
1
1
4
1
1
0
1
1
1
1
2
1
2
2
3
1
1
1
3
4
6
3
1
2
4
3
2
3
2
12
3
4
1
2
2
2
6
2
3
4
4
2
2
3
2
3
0
2
5
2
3
1
2
2
2
1
2
1
3
2
1
6
2
1
2
2
3
3
2
3
1
2
2
1
4
2
3
4
1
2
3
3
1
1
3
3
2
1
1
3
2
3
3
2
2
2
2
5
2
1
2
4
1
3
0
2
2
4
3
2
4
2
2
18
3
2
3
2
1
1
1
1
3
1
3
2
1
1
2
1
3
2
1
3
2
6
0
2
2
2
5
2
2
5
2
1
1
2
1
10
2
4
1
1
4
3
3
2
1
1
1
1
1
2
2
2
1
2
1
3
1
2
4
0
1
4
1
6
3
2
2
1
2
3
11
2
2
2
5
2
1
2
2
3
5
3
3
2
2
1
2
5
2
4
5
5
8
1
2

In [23]:
def count_x(lst, x):
    count = 0
    for ele in lst:
        if (ele == x):
            count = count + 1
    return count

count_x(methods_list,[])

101

# create and export clean dataframe

In [24]:
scrape_df['science_direct_methods'] = methods_list

C:\Users\Joe Z\anaconda3\envs\scraper\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
explode_df = pd.DataFrame(scrape_df['science_direct_methods'].tolist())

In [35]:
explode_df = explode_df.astype(str).apply(lambda x: x.str[:30000])

In [36]:
explode_df['pmid'] = scrape_df['pmid'].to_list()

In [41]:
explode_df.replace('None', np.nan, inplace=True)

In [42]:
explode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6127 entries, 0 to 6126
Data columns (total 36 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6026 non-null   object
 1   1       4322 non-null   object
 2   2       2439 non-null   object
 3   3       1331 non-null   object
 4   4       732 non-null    object
 5   5       459 non-null    object
 6   6       280 non-null    object
 7   7       179 non-null    object
 8   8       111 non-null    object
 9   9       75 non-null     object
 10  10      51 non-null     object
 11  11      36 non-null     object
 12  12      23 non-null     object
 13  13      20 non-null     object
 14  14      15 non-null     object
 15  15      14 non-null     object
 16  16      14 non-null     object
 17  17      11 non-null     object
 18  18      8 non-null      object
 19  19      8 non-null      object
 20  20      7 non-null      object
 21  21      7 non-null      object
 22  22      7 non-null      

In [43]:
explode_df.to_csv('output/01b_science_direct_methods_explode.csv')